In [808]:
import re
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from konlpy.tag import Okt
from ckonlpy.tag import Twitter as Okt
from ckonlpy.tag import Postprocessor
from collections import Counter

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

path = '/VSC/JJaemni/CSV/'
df = pd.read_csv(path + 'wanted_jobplanet_2.csv')

# import matplotlib.pyplot as plt

# # 가능한 font list 확인
# import matplotlib.font_manager as fm
# f = [f.name for f in fm.fontManager.ttflist]
# print(f)

# # 확인 이후
# plt.rc('font', family='Malgun Gothic')

# **불용어 제거 및 표준화**
- 한 행 씩 보면서, 불용어 표준화 진행을 위한 전처리를 먼저 수행

In [782]:
req = df.Requirement.tolist()

# 한글, 영문 대소문자, 숫자, 공백을 제외한 문자를 제거
req2 = []
for idx in range(0, df.shape[0]):
    str = req[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    req2.append(new_str4)
    # print(f'{idx}. {req2[idx]}')
req2_df = pd.DataFrame({'req': req2})


# Kor, Eng 분리
eng_list = []
kor_list = []
for idx in range(0, req2_df.shape[0]):
    text = req2_df.req[idx]
    eng_text = ' '.join(re.findall(r'[A-Za-z]+', text))
    kor_text = re.sub(r'[A-Za-z]+', '', text).strip()
    eng_list.append(eng_text)
    kor_list.append(kor_text)


# 영어 띄어쓰기 기준으로 split
eng_split_list = []
for eng in eng_list:
    spl = eng.split()
    eng_split_list.append(spl)
# for idx, eng in enumerate(eng_split_list):
    # print(f'{idx}. {eng}')


# 영어 소문자로 통일
eng_lower = []
for sub in eng_split_list:
    sublist = []
    for eng in sub:
        eng_ = eng.lower()
        sublist.append(eng_)
    eng_lower.append(sublist)
# for idx, eng in enumerate(eng_lower):
    # print(f'{idx}. {eng}')
print(len(eng_lower))

181


## **Requirement: Kor**

### **명사 추가, 단어 치환, 불용어 처리**
- `customized_konlpy`

In [765]:
logging.getLogger('konlpy').setLevel(logging.ERROR)
okt = Okt()

# 명사 추가
custom_nouns = ['알고리즘', '통계분석', '인공지능', '머신러닝', '블록체인', '실무경험', '모델링', '개발역량', '인사이트', '딥러닝', '프레임워크', '시각화', '프로젝트', '포트폴리오', '사이언티스트', '개인화', '프로그래밍', '협업', '의사소통', '컴퓨터공학', '산업공학', '자연어', '논문', '핸들링', '전처리', '수리적', '통계적', '협력','커뮤니케이션', '통계학과', '수학', '데이터사이언티스트', '소통', '시계열', '도메인', '문제정의', '가설검증','파이프라인', '책임감', '문제', '레이블링', '라벨링', '가설', '해결책', '기초통계학', '의사결정', '논리적', '분석적', '데이터베이스', '보고서', '연구', '리포트', '성실', '통계학', '비즈니스', '최신', '주도적', '비정형', '정형', '문서', '자기소개서', '대시보드', '검증', '빅데이터', '수집', '설득', '리포팅', '지식', '분석','리딩', '데이터처리', '데이터분석', '웹분석', '웹', '영어', '꼼꼼', '끈질', '끈기', '문서화', '수학적', '가공', '정제', '리텐션', '코호트', '엔지니어링', '구글스프레드시트', '엑셀', '인텔리전스', '인과추론', '로그', '구조적','적응', '파워포인트', '오너십', '추출', '악성코드', '취약점', '주도적', '연구개발', '컨설팅', '코드', '테스트','제조', '스마트팩토리', '솔루션', '시스템', '공정관리', '생산관리', '품질관리', '설비관리', '이해도', '백엔드', '클라우드', '어플리케이션', '도전의식', '열정', '스크래핑', '앱서비스', '적극적', '소프트웨어', '고가용성', '내고장성', '다이어그램', '스키마', '설계', '구현', '경험', '기반', '구축', '엔지니어', '자동화', '개발경험','플랫폼', '거버넌스', '기술통계', '경영학', '경영정보학', '쿼리', '백업', '모니터링', '장애처리', '최적화', '정보보호', '리눅스', '금융공학', '물리학', '마트', '컨테이너', '튜닝', '개발자', '트러블슈팅', '실시간', '인프라', '퀄리티', '전산', '이슈', '웹크롤링', '오케스트레이션', '워크플로우', '텍스트', '비동기처리', '네트워크', '웹서비스', '하둡', '프로그램', '하이퍼', '캐주얼', '게임', '프로세싱', '프로세스', '아키텍처', '서버', '호기심', '프로토타이핑', '프레젠테이션', '스크립팅', '스트림', '접근의지', '분산', '클러스터', '포트폴리오', '배치', '레이크하우스', '스트리밍', '라이브러리', '비즈니스', '신호처리', '프로덕트', '백앤드']
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

# 단어 치환
replace = {'컨테이너화': '컨테이너', '장애': '장애처리', '백앤드': '백엔드', '프로덕트': '프로덕트 데이터 분석', '이미지': '이미지데이터', '인과': '인과추론', '레이크하우스': '데이터 레이크하우스', '분산': '분산 처리 시스템', '접근의지': '도전적', '서비스': '서비스 배포 운영', '프레젠테이션': '프레젠테이션 기술', '플랫폼': '플랫폼 구축', '솔루션': '데이터 솔루션', '서버': '서버 설계', '프로세싱': '데이터 프로세싱', '하이퍼': '하이퍼 캐주얼 게임', '컴퓨터': '컴퓨터공학', '텍스트': '텍스트데이터', '스크래핑': '웹크롤링', '이슈': '이슈 해결', '전산': '전산 지식', '퀄리티': '높은 코드 퀄리티', '인프라': '인프라 구축 운영', '실시간': '실시간 데이터 처리', '빅': '빅데이터', '마트': '데이터 마트 구축 운영', '의사': '커뮤니케이션', '의사결정': '데이터 기반 의사결정', '앱서비스': '데이터 어플리케이션 서비스', '도전': '도전적', '도전의식': '도전적', '열정': '열정적', '어플리케이션': '어플리케이션 개발', '클라우드': '클라우드 경험', '테스트': '테스트 설계 분석', '코드': '코드 해석', '취약점': '취약점 분석', '악성코드': '악성코드 분석', '적응': '적응 능력', '구조적': '구조적 사고', '로그': '데이터 로그 설계 분석', '인텔리전스': '비즈니스 인텔리전스', '엔지니어링': '데이터 엔지니어링', '리텐션': '리텐션 분석', '코호트': '코호트 분석', '정제': '데이터 정제', '가공': '데이터 가공', '수학적': '수리적 지식', '빅데이터': '빅데이터 분석', '문서화': '리포트 작성', '끈질': '끈질김', '꼼꼼': '꼼꼼함', '데이터분석': '데이터 분석', '데이터처리': '데이터 처리', '리딩': '리딩 능력', '지식': '도메인 지식', '리포팅': '리포팅 능력', '수집': '데이터 수집', '비즈니스': '문제 정의 및 해결', '대시보드': '대시보드 구축', '프로그래밍': '프로그래밍 언어', '자기소개서': '포트폴리오', '정형': '정형데이터', '비정형': '비정형데이터', '해결책': '해결책 탐색', '최신': '최신 기술 연구', '통계학': '통계적 지식', '성실': '성실함', '리포트': '리포트 작성', '보고서': '리포트 작성', '수립': '가설 수립 및 검증', '검증': '가설 수립 및 검증', '논리적': '논리적 사고', '분석적': '분석적 사고', '기초통계학': '통계적 지식', '가설': '가설 수립 및 검증', '가설검증': '가설 수립 및 검증', '레이블링': '데이터 레이블링', '라벨링': '데이터 레이블링', '시계열': '시계열 데이터', '사이언스': '데이터 사이언스', '분석': '데이터 분석', '모델링': '데이터 모델링', '인사이트': '인사이트 도출', '프로젝트': '프로젝트 경험', '사이언티스트': '데이터 사이언티스트', '개인화': '개인화 추천', '자연어': '자연어 처리', '실무': '실무경험', '시각화': '데이터 시각화', '논문': '논문 구현', '전처리': '데이터 전처리', '수리적': '수리적 지식', '통계적': '통계적 지식', '의사소통': '커뮤니케이션', '데이터사이언티스트': '데이터 사이언티스트', '소통': '커뮤니케이션', '도메인': '도메인 지식', '추출': '데이터 추출', '핸들링': '데이터 핸들링', '파이프라인': '데이터 파이프라인 구축', '문제': '문제 정의 및 해결', '문제정의': '문제 정의 및 해결'}

# 불용어 처리
stopwords = {'박사', '컨셉', '자신', '데이터', '배민', '다양', '설득', '한글', '이신', '각각', '어간', '대졸', '아야', '피드백', '액션', '신용', '참조', '결격', '기술', '패키지', '대면', '구성원', '광범위', '아키텍처', '정량', '센터', '계열', '체계', '주어', '기회', '거래', '더욱', '통계', '오픈', '캐주얼', '업무', '서면', '면서', '평가', '요구', '파일', '자유도', '처리', '사고력', '특성', '테이블', '축하', '금융기관', '퍼널', '레벨', '혁신', '리드', '디바이스', '발생', '대학원', '마인드', '대용', '상의', '경우', '저축은행', '사용자', '제안', '필수', '분야', '수무', '집중', '우리', '용해', '기보', '차량', '해당', '숙자', '건축', '실제', '해외여행', '생애', '파악', '크립', '고민', '디자인', '외산', '소스', '자기', '과학', '학위', '연동', '운영', '자격', '확장', '거나', '적용', '이해도', '관찰', '하나', '진의', '질적', '루시', '특별', '내외', '그림', '구현', '기획', '일련', '여행', '요건', '원리', '라이프', '설명', '전반', '직접', '편향', '환경', '단계', '대리', '지금', '팀원', '로서', '개선', '요하', '문화', '의료', '통해', '정확', '정의', '통합', '이수', '개발자', '아이디어', '수행', '사항', '인원', '임팩트', '드립', '국내', '분분', '필요', '이력', '나은', '자율', '이후', '유관', '서브', '용어', '개발', '어도', '다른', '학과', '이학', '모바일', '구글', '능숙', '대학교', '표현', '소유자', '양식', '초대', '환영', '전공자', '매우', '자격증', '이면', '설치', '부탁', '절차', '다차원', '규제', '지자', '비전문가', '이력서', '스킬', '따라서', '함수', '케이스', '대학', '기반', '유리', '제하', '아래', '결과', '체인', '간의', '셨다', '대해', '기법', '조인', '전달', '작업', '자세', '담당자', '졸업', '적절', '보신', '시드', '이상인', '물리', '과제', '예측', '학력', '명확', '유하', '영해', '수행자', '무관', '스토어', '시니어', '영역', '신청', '동기', '학사', '대한', '암호', '의지', '수준', '상황', '이나', '고도화', '실력', '실험', '희망', '가치관', '경험', '본적', '진단', '이해', '기타', '병무청', '가독성', '유경', '사유', '버전', '수요', '사업', '어려움', '진행', '실행', '주요', '산업', '국내외', '구사', '관점', '라운드', '언어', '기초', '연구', '출장', '내용', '복잡', '실패', '추세', '모시', '마케팅', '전체', '성능', '부하', '테스', '찾기', '학습', '게임', '구성', '지원', '전문', '익숙', '강의', '목적', '설계', '대안', '캠페인', '컨설팅', '간결', '한가지', '스택', '사원', '선호', '전략', '요구사항', '대하', '제도', '이상보', '멀티', '비롯', '타부', '대규모', '조건', '추가', '임해', '동형', '고객', '숫자', '능통', '제한', '독해', '드릴', '패턴', '예정자', '담당', '분주', '변화', '갖은', '공유', '모니터링', '서빙', '클라이언트', '엔지니어', '제거', '핵심', '박사학위', '지원동', '개인', '이커머스', '초봉', '자유', '선하', '해석', '품질', '특허', '기간', '전하', '누구', '설정', '물류', '활용', '해결', '앤드', '상이', '제시', '도록', '브랜드', '검색', '시도', '중요성', '응용', '말로', '간호', '전공', '로부터', '석사', '선착순', '탐색', '결하', '상용', '위해', '아하', '향상', '리하', '구축', '할하', '가시', '영문', '직무', '정적', '신규', '양상', '기본', '사람', '발굴', '모형', '방법', '능력', '러닝', '관리', '개념', '프로세스', '대상', '제외', '투명', '자유자', '기고', '이론', '디자이너', '웹사이트', '시리즈', '최전선', '미만', '포함', '사이클', '튜닝', '나신', '영어', '프로그램', '지표', '논리', '원활', '성공', '도입', '이상', '방향', '존중', '로우', '스타트업', '경영', '연구원', '스펙', '면접', '오픈소스', '신분', '가치', '편입', '추천', '가지', '상급', '형성', '당하', '근무', '조직', '지방', '스스로', '의미', '해외', '제품', '수록', '취득', '게임기', '생명', '제출', '공학', '동영상', '프로덕션', '트렌드', '부동산', '원본', '래서', '자체', '문서', '근거', '지향', '동료', '흥미', '수학', '신입', '긴밀', '사용', '개편', '바탕', '단독', '방식', '유용', '효율', '어보', '공정', '단순화', '도구', '구조', '효과', '리뷰', '불가', '체감', '정보화', '관심', '숙련', '연하', '가능', '방법론', '이공', '문학사', '작성', '반복', '유수', '까지', '사내', '나가시', '추적', '중심', '추구', '분기', '이름', '형식', '상응', '소지', '결함', '두려움', '사고', '유연', '기업', '임상시험', '분드', '역량', '라면', '출하', '인정', '상호', '서류', '마감', '연봉', '통한', '어요', '본질', '어진', '발표', '부터', '자동화', '이용', '검수', '투자', '합리', '동작', '이해력', '모델', '보고', '아직', '과정', '교보', '플레이어', '단순', '우수', '스퀘어', '경력', '데이터마트', '여러', '아래아한글', '상품', '관련', '이하', '기준', '예정', '우대', '성향', '의견', '도출', '반적', '고급', '보유', '사이트', '본인', '성과', '영상', '정보', '홈택스', '여러가지', '굳이', '시간', '의학', '자부', '공감', '활동', '과장', '플래닛', '성장', '최소', '정책', '지니', '부서', '형태', '방향성', '깊이', '커브', '생각', '크루'}


postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, len(kor_list)):
    elem = kor_list[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        if len(word) > 1:
            result.append(word)
    results.append(result)
    print(f'{idx}. {result}')

0. ['시계열 데이터', '데이터 분석', '논문 구현', '알고리즘', '데이터 분석', '통계분석', '인공지능', '머신러닝']
1. ['데이터 사이언스', '블록체인', '데이터 분석', '머신러닝', '데이터 분석']
2. ['데이터 사이언스', '실무경험', '데이터 분석', '머신러닝', '데이터 분석', '문제 정의 및 해결', '알고리즘', '논문 구현', '데이터 분석', '블록체인', '비지니스', '실무경험']
3. ['데이터 모델링', '개발역량', '데이터 분석', '데이터 분석', '데이터 분석', '문제 정의 및 해결', '인사이트 도출']
4. ['데이터 모델링']
5. ['딥러닝', '프레임워크', '배포', '데이터 분석', '데이터 시각화', '적극적', '커뮤니케이션', '주도적', '개발경험', '협업', '프로젝트 경험', '협업', '포트폴리오']
6. ['데이터 사이언티스트', '데이터 모델링', '프로젝트 경험', '실무경험', '개인화 추천', '데이터 모델링', '도메인 지식', '프로그래밍 언어', '수리적 지식', '협업', '커뮤니케이션']
7. ['컴퓨터공학', '산업공학', '도메인 지식', '데이터 모델링', '알고리즘', '자연어 처리', '가설 수립 및 검증', '최적화', '데이터 분석', '데이터 분석', '데이터 모델링', '문제 정의 및 해결']
8. ['프로젝트 경험', '실무경험', '서비스 배포 운영', '데이터 분석', '머신러닝', '주도적', '끈질김', '인사이트 도출', '이슈 해결', '문제 정의 및 해결']
9. ['컴퓨터공학', '산업공학', '데이터 분석', '데이터 시각화', '데이터 분석', '열정적']
10. ['실무경험', '머신러닝', '프로젝트 경험', '리딩 능력', '논문 구현', '서비스 배포 운영', '문제 정의 및 해결', '커뮤니케이션', '협업', '문제 정의 및 해결']
11. ['데이터 추출', '데이터 핸들링', '데이터 분석', '프로젝

In [767]:
print(results)
print(len(results))

[['시계열 데이터', '데이터 분석', '논문 구현', '알고리즘', '데이터 분석', '통계분석', '인공지능', '머신러닝'], ['데이터 사이언스', '블록체인', '데이터 분석', '머신러닝', '데이터 분석'], ['데이터 사이언스', '실무경험', '데이터 분석', '머신러닝', '데이터 분석', '문제 정의 및 해결', '알고리즘', '논문 구현', '데이터 분석', '블록체인', '비지니스', '실무경험'], ['데이터 모델링', '개발역량', '데이터 분석', '데이터 분석', '데이터 분석', '문제 정의 및 해결', '인사이트 도출'], ['데이터 모델링'], ['딥러닝', '프레임워크', '배포', '데이터 분석', '데이터 시각화', '적극적', '커뮤니케이션', '주도적', '개발경험', '협업', '프로젝트 경험', '협업', '포트폴리오'], ['데이터 사이언티스트', '데이터 모델링', '프로젝트 경험', '실무경험', '개인화 추천', '데이터 모델링', '도메인 지식', '프로그래밍 언어', '수리적 지식', '협업', '커뮤니케이션'], ['컴퓨터공학', '산업공학', '도메인 지식', '데이터 모델링', '알고리즘', '자연어 처리', '가설 수립 및 검증', '최적화', '데이터 분석', '데이터 분석', '데이터 모델링', '문제 정의 및 해결'], ['프로젝트 경험', '실무경험', '서비스 배포 운영', '데이터 분석', '머신러닝', '주도적', '끈질김', '인사이트 도출', '이슈 해결', '문제 정의 및 해결'], ['컴퓨터공학', '산업공학', '데이터 분석', '데이터 시각화', '데이터 분석', '열정적'], ['실무경험', '머신러닝', '프로젝트 경험', '리딩 능력', '논문 구현', '서비스 배포 운영', '문제 정의 및 해결', '커뮤니케이션', '협업', '문제 정의 및 해결'], ['데이터 추출', '데이터 핸들링', '데이터 분석', '프로젝트 경험', '데이터 전처리', '데이터 분석'

In [769]:
# 리스트 중복 요소 제거

Req_kor_final = []
for idx in range(0, len(kor_list)):
    a = set(results[idx])
    b = list(a)
    Req_kor_final.append(b)
print(Req_kor_final)
print(len(Req_kor_final))

[['데이터 분석', '시계열 데이터', '알고리즘', '논문 구현', '인공지능', '머신러닝', '통계분석'], ['머신러닝', '블록체인', '데이터 분석', '데이터 사이언스'], ['데이터 분석', '알고리즘', '머신러닝', '실무경험', '비지니스', '문제 정의 및 해결', '데이터 사이언스', '블록체인', '논문 구현'], ['데이터 분석', '문제 정의 및 해결', '개발역량', '데이터 모델링', '인사이트 도출'], ['데이터 모델링'], ['데이터 시각화', '데이터 분석', '프레임워크', '주도적', '포트폴리오', '적극적', '딥러닝', '협업', '배포', '개발경험', '프로젝트 경험', '커뮤니케이션'], ['도메인 지식', '실무경험', '수리적 지식', '데이터 사이언티스트', '개인화 추천', '협업', '프로그래밍 언어', '데이터 모델링', '프로젝트 경험', '커뮤니케이션'], ['최적화', '데이터 분석', '알고리즘', '도메인 지식', '자연어 처리', '가설 수립 및 검증', '컴퓨터공학', '문제 정의 및 해결', '산업공학', '데이터 모델링'], ['이슈 해결', '데이터 분석', '주도적', '머신러닝', '실무경험', '문제 정의 및 해결', '서비스 배포 운영', '프로젝트 경험', '끈질김', '인사이트 도출'], ['데이터 시각화', '데이터 분석', '열정적', '컴퓨터공학', '산업공학'], ['머신러닝', '실무경험', '리딩 능력', '문제 정의 및 해결', '협업', '서비스 배포 운영', '프로젝트 경험', '커뮤니케이션', '논문 구현'], ['데이터 핸들링', '데이터 분석', '데이터 전처리', '데이터 추출', '데이터 마트 구축 운영', '프로젝트 경험'], ['이슈 해결', '데이터 분석', '시스템', '알고리즘', '도메인 지식', '머신러닝', '수리적 지식', '문제 정의 및 해결', '서비스 배포 운영', '데이터 모델링', '커뮤니케이션', '통계적 지식', 

In [774]:
Req_kor_final_df = pd.DataFrame({'Requierement_KOR': Req_kor_final})
Req_kor_final_df.Requierement_KOR = Req_kor_final_df.Requierement_KOR.apply(lambda x: ', '.join(x))
Req_kor_final_df.to_csv(path + 'Requirement_Kor.csv', index=False)

In [740]:
for key, val in replace.items():
    print(key, end=', ')

컨테이너화, 장애, 백앤드, 프로덕트, 이미지, 인과, 레이크하우스, 분산, 접근의지, 서비스, 프레젠테이션, 플랫폼, 솔루션, 서버, 프로세싱, 하이퍼, 컴퓨터, 텍스트, 스크래핑, 이슈, 전산, 퀄리티, 인프라, 실시간, 빅, 마트, 의사, 의사결정, 앱서비스, 도전, 도전의식, 열정, 어플리케이션, 클라우드, 테스트, 코드, 취약점, 악성코드, 적응, 구조적, 로그, 인텔리전스, 엔지니어링, 리텐션, 코호트, 정제, 가공, 수학적, 빅데이터, 문서화, 끈질, 꼼꼼, 데이터분석, 데이터처리, 리딩, 지식, 리포팅, 수집, 비즈니스, 대시보드, 프로그래밍, 자기소개서, 정형, 비정형, 해결책, 최신, 통계학, 성실, 리포트, 보고서, 수립, 검증, 논리적, 분석적, 기초통계학, 가설, 가설검증, 레이블링, 라벨링, 시계열, 사이언스, 분석, 모델링, 인사이트, 프로젝트, 사이언티스트, 개인화, 자연어, 실무, 시각화, 논문, 전처리, 수리적, 통계적, 의사소통, 데이터사이언티스트, 소통, 도메인, 추출, 핸들링, 파이프라인, 문제, 문제정의, 

## **Requirement: Eng**

In [786]:
for idx in range(0, len(eng_lower)):
    print(f'{idx}. {eng_lower[idx]}')

0. ['tensorflow', 'pytorch', 'r', 'tableau']
1. []
2. []
3. ['ml', 'agile', 'scrum', 'sprint', 'squad']
4. ['tableau', 'ml', 'pytorch', 'tensorflow', 'keras', 'aws', 'gcp']
5. ['data', 'centric', 'ai', 'd', 'd', 'linux']
6. ['data', 'science', 'b', 'c', 'spark', 'numpy', 'pandas', 'spark', 'ml', 'dl', 'scikit', 'learn', 'pytorch', 'tensorflow', 'aws']
7. []
8. ['a', 'b', 'customer', 'lifetime', 'value', 'retention', 'cohort', 'analysis', 'churn', 'prediction']
9. []
10. ['ai']
11. ['deep', 'learning', 'linux', 'shell', 'script', 'java', 'python', 'data', 'manipulation', 'algorithms', 'customizing', 'data', 'visualization', 'tool', 'tableau', 'spotfire', 'd', 'js']
12. ['end', 'to', 'end']
13. ['ai']
14. ['aws', 'gcp', 'tensorflow', 'keras', 'pytorch', 'deep', 'learning', 'framework', 'spark', 'ray', 'kaggle']
15. []
16. ['aws', 'gcp', 'ml', 'pm', 'kubernetes', 'airflow']
17. ['phd', 'degree', 'in', 'a', 'quantitative', 'discipline', 'ability', 'to', 'manage', 'multiple', 'projects', 'a

- 17행의 공고내용은 영어였음

### **불용어 처리**

In [807]:
a = ' '.join(eng_lower[17])
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(a)

results = []
for token in word_tokens:
    if token not in stop_words:
        results.append(token)
print(word_tokens)
print(results)

['phd', 'degree', 'in', 'a', 'quantitative', 'discipline', 'ability', 'to', 'manage', 'multiple', 'projects', 'at', 'the', 'same', 'time', 'while', 'paying', 'attention', 'to', 'detail', 'and', 'translate', 'analysis', 'results', 'into', 'business', 'recommendations', 'proven', 'ability', 'to', 'own', 'projects', 'end', 'to', 'end', 'even', 'when', 'you', 'have', 'to', 'get', 'creative', 'you', 'do', 'more', 'than', 'completing', 'delegated', 'tasks', 'demonstrated', 'leadership', 'and', 'self', 'direction', 'willingness', 'to', 'both', 'teach', 'others', 'and', 'learn', 'new', 'techniques']
['phd', 'degree', 'quantitative', 'discipline', 'ability', 'manage', 'multiple', 'projects', 'time', 'paying', 'attention', 'detail', 'translate', 'analysis', 'results', 'business', 'recommendations', 'proven', 'ability', 'projects', 'end', 'end', 'even', 'get', 'creative', 'completing', 'delegated', 'tasks', 'demonstrated', 'leadership', 'self', 'direction', 'willingness', 'teach', 'others', 'lear

- 영어는 일단 `StackTool` 정리한 후에 다시 처리하기

---

In [784]:
# 한글, 영문 대소문자, 숫자, 공백을 제외한 문자를 제거
prefer = df.Preference.tolist()

prefer2 = []
for idx in range(0, df.shape[0]):
    str = prefer[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    prefer2.append(new_str4)
    # print(f'{idx}. {prefer2[idx]}')
prefer2_df = pd.DataFrame({'prefer': prefer2})


# Kor, Eng 분리
eng_list = []
kor_list = []
for idx in range(0, prefer2_df.shape[0]):
    text = prefer2_df.prefer[idx]
    eng_text = ' '.join(re.findall(r'[A-Za-z]+', text))
    kor_text = re.sub(r'[A-Za-z]+', '', text).strip()
    eng_list.append(eng_text)
    kor_list.append(kor_text)


# 영어 띄어쓰기 기준으로 split
eng_split_list = []
for eng in eng_list:
    spl = eng.split()
    eng_split_list.append(spl)
# for idx, eng in enumerate(eng_split_list):
    # print(f'{idx}. {eng}')


# 영어 소문자로 통일
eng_lower = []
for sub in eng_split_list:
    sublist = []
    for eng in sub:
        eng_ = eng.lower()
        sublist.append(eng_)
    eng_lower.append(sublist)
# for idx, eng in enumerate(eng_lower):
    # print(f'{idx}. {eng}')
print(len(eng_lower))

181


## **Preference: Kor**

### **명사 추가, 단어 치환, 불용어 처리**
- `customized_konlpy`

# **Preference**

In [992]:
logging.getLogger('konlpy').setLevel(logging.ERROR)
okt = Okt()

# 명사 추가
custom_nouns = [
    '가상관련', '머신러닝', '모델링', '분석', '블록체인', '암호화폐', '빅데이터', '통계분석', '머신러닝', '영어', '의사소통', 
    '수학', '전산', '시각화', '프레임워크', '클라우드', '인공지능', '컴퓨터공학', '데이터분석', '개발경험', '자율주행', 
    '통계학과', '수학과', '컴퓨터과학', '모델링', '논리적', '사이언티스트', '가설적', '구조적', '사이언스', '마이닝', 
    '통계학', '정형', '고객', '열정적', '커뮤니케이션', '도메인', '데이터마이닝', '논문', '베포', '프로젝트', '산업공학', 
    '경영정보학', '경영학', '경제학', '딥러닝', '프레임워크', '프로그래밍', '프로덕트', '클라우드', '분산', '입상자', '리딩', 
    '소통', '시스템', '실무', '테스트', '의견', '비정형', '빅데이터', '가공', '이미지', '자연어', '인사이트', '자기주도적', 
    '주도적', '도전정신', '체계적', '유관부서', '책임감', '플랫폼', '포트폴리오', '크롤링', '수집', '엔지니어링', '인프라', 
    '딥러닝', '시계열', '레이블링', '라벨링', '대시보드', '인과추론', '분산처리', '데이터베이스', '문제해결', '이커머스', 
    '프리랜서', '행동', '파이프라인', '정보처리', '이공계열', '기술', '챗봇', '프리젠테이션', '꼼꼼', '데이터분석준전문가', 
    '분석기사', '협업', '실무매니징', '휴먼매니징', '웹사이트', '시스템', '전처리', '품질', '프로세스', '리터러시', '컨테이너',
    '데이터파이프라인', '빅데이터플랫폼', '쿼리', '스크립트', '시공간', '웹로그', '웹페이지', '준전문가', '분석적', 
    '트러블슈팅', '핀테크', '해킹대회', '정보보호', '자격증', '비즈니스', '모델', '머신비전', '프로세싱', '최적화', '리포팅',
    '웨어하우스', '백엔드', '워크플로우', '레이크', '마트', '애플리케이션', '정보처리기사', '서비스', '파이썬', '병렬',
    '프론트엔드', '긍정적', '고가용성', '정보보안기사', '정보관리기술사', '회화', '적극적', '리눅스', '영어회화',
    '오케스트레이션', '서버', '데이터마트', '온프레미스', '컴퓨터비전', '해결', '직무', '웨어하우징', '분산처리', '정형',
    '비정형', '표준화', '온체인', '로그분석', '엑셀', '자바', '입상', '엔진', '테이블', '지향적'
]
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

# 단어 치환
replace = {
    
    '지향적': '목표지향적',
    '테이블': '테이블 포맷',
    '엔진': '엔진 경험',
    '입상': '대회 입상자',
    '자바': '자바스크립트',
    '온체인': '온체인데이터',
    '비정형': '비정형데이터',
    '정형': '정형데이터',
    '분산처리': '분산 처리 시스템',
    '웨어하우징': '빅데이터 웨어하우징',
    '직무': '직무경험',
    '프로세싱': '데이터 프로세싱',
    '표준화': '데이터 표준화',
    '열정': '열정적',
    '해결': '문제해결능력',
    '데이터마트': '데이터 마트 구축 및 운영',
    '서버': '서버 개발',
    '트러블 슈팅': '트러블슈팅',
    '영어회화': '영어 커뮤니케이션',
    '회화': '영어 커뮤니케이션',
    '병렬': '병렬 처리 프로그래밍',
    '데이터웨어하우스': '데이터 웨어하우스',
    '서비스': '서비스 구축 및 운영',
    '애플리케이션': '데이터 애플리케이션',
    '마트': '데이터 마트 구축 및 운영',
    '레이크': '데이터 레이크',
    '정보처리': '정보처리기사',
    '리포팅': '리포팅 능력',
    '웨어하우스': '데이터 웨어하우스',
    '머신비전': '머신비전 이미지 프로세싱',
    '모델': '모델 구축',
    '자격증': '자격증 보유',
    '가설': '가설 수립 및 검증',
    '러닝': '머신러닝',
    '머신': '머신러닝',
    '분석적': '분석적 사고',
    '수상': '대회 입상자',
    '대회': '대회 입상자',
    '준전문가': '데이터분석준전문가',
    '웹로그': '웹로그 분석', 
    '웹페이지': '웹페이지 분석',
    '시공간': '시공간데이터',
    '스크립트': '스크립트 경험',
    '쿼리': '쿼리 튜닝',
    '빅데이터플랫폼': '빅데이터 플랫폼 구축',
    '데이터파이프라인': '데이터 파이프라인',
    '컨테이너': '컨테이너 경험',
    '고가용성': '고가용성 데이터 플랫폼 설계',
    '리터러시': '데이터 리터러시',
    '품질': '데이터 품질 관리',
    '전처리': '데이터 전처리',
    '시스템': '시스템 경험',
    '웹사이트': '웹사이트 경험',
    '분석기사': '빅데이터분석기사',
    '꼼꼼': '꼼꼼함',
    '챗봇': '챗봇 설계 및 개발',
    '파이프라인': '데이터 파이프라인',
    '문제해결': '문제해결능력',
    '분산처리': '분산 처리 시스템',
    '인과': '인과추론',
    '영상': '영상데이터',
    '대시보드': '대시보드 경험',
    '레이블링': '데이터 레이블링',
    '라벨링': '데이터 레이블링',
    '시계열': '시계열데이터',
    '논문': '논문 출판 및 논문 구현',
    '엔지니어링': '데이터 엔지니어링',
    '수집': '데이터 수집',
    '크롤링': '웹크롤링', 
    '플랫폼': '플랫폼 구축',
    '체계적': '체계적 사고',
    '가상관련': '가상 데이터', 
    '분석': '데이터 분석', 
    '의사소통': '커뮤니케이션', 
    '수학': '수학과', 
    '통계': '통계 분석', 
    '전산': '전산학과', 
    '시각화': '데이터 시각화', 
    '데이터분석': '데이터 분석', 
    '논리적': '논리적 사고', 
    '처리': '데이터 처리', 
    '가설적': '가설 수립 및 검증', 
    '구조적': '구조적 사고', 
    '사이언스': '데이터 사이언스', 
    '마이닝': '데이터 마이닝', 
    '통계학': '통계학과', 
    '정형': '정형데이터', 
    '고객': '고객데이터', 
    '도메인': '도메인 지식', 
    '데이터마이닝': '데이터 마이닝', 
    '배포': '서비스 배포', 
    '영어': '영어 커뮤니케이션', 
    '프로그래밍': '프로그래밍 언어', 
    '프로덕트': '프로덕트 개발', 
    '클라우드': '클라우드 경험', 
    '분산': '분산 처리 시스템', 
    '입상자': '대회 입상자', 
    '리딩': '리딩 경험', 
    '프로젝트': '프로젝트 경험', 
    '소통': '커뮤니케이션', 
    '실무': '실무경험', 
    '테스트': '테스트 계획 및 결과 분석', 
    '의견': '의견 제시 경험', 
    '비정형': '비정형데이터', 
    '가공': '데이터 가공', 
    '이미지': '이미지데이터', 
    '자연어': '자연어 처리', 
    '인사이트': '인사이트 도출', 
    '자기주도적': '주도적'
}

# 불용어 처리
stopwords = {}


postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, len(kor_list)):
    elem = kor_list[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        if len(word) > 1:
            result.append(word)
    results.append(result)
    print(f'{idx}. {result}')

c:\Users\LG\anaconda3\envs\crawling\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


0. ['기상', '관련', '데이터', '경험', '머신러닝', '관련', '모델링', '도구', '경험', '데이터', '데이터 분석', '경험']
1. ['블록체인', '암호화폐', '관련', '기업', '근무', '경력', '빅데이터', '관련', '기업', '근무', '경력', '관련', '대회 입상자', '대회 입상자', '대회 입상자', '경력', '통계분석', '머신러닝', '관련', '분야', '박사', '학위', '소지', '상위', '수준', '영어 커뮤니케이션', '커뮤니케이션', '문서', '작성', '능력']
2. ['블록체인', '암호화폐', '관련', '기업', '근무', '경력', '빅데이터', '관련', '기업', '근무', '경력', '관련', '대회 입상자', '대회 입상자', '대회 입상자', '경력', '통계분석', '머신러닝', '관련', '분야', '박사', '학위', '소지', '상위', '수준', '영어 커뮤니케이션', '커뮤니케이션', '문서', '작성', '능력']
3. ['관련', '학과', '수학과', '통계 분석', '전산학과', '데이터', '데이터 분석', '전공자', '우대', '관련', '서비스 구축 및 운영', '개발', '운영', '경험', '우대', '업무', '환경', '경험', '우대']
4. ['데이터', '데이터 시각화', '도구', '사용', '가능', '프레임워크', '활용', '가능', '클라우드 경험', '사용', '경험']
5. ['관련', '분야', '석사', '박사', '학위', '소지', '인공지능', '컴퓨터공학', '통계 분석', '데이터 분석', '대한', '이해도', '관련', '개발경험', '자율주행', '데이터', '활용', '경험', '이미지데이터', '라이', '레이더', '등등', '데이터', '경험', '기반', '개발', '경험']
6. ['경험', '이면', '우대', '사항', '관련', '전공', '통계학과', '수학과', '컴퓨터과학', '석사'

- 일어나서 우대사항 불용어 추가하기